# Simple example run for the use of the VariableSPL process

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xsimlab as xs
import xarray as xr
%load_ext xsimlab.ipython


We import the basic model from FastScape and modify it by replacing the SPL process by the VariableSPL process and dropping the diffusion process

In [ ]:
from fastscape.models import basic_model
from VariableSPL import VariableSPL
spl_model = basic_model.update_processes({'spl': VariableSPL}).drop_processes(('diffusion'))

spl_model.visualize()

We create a local cluster to run a large number of model runs in bacth mode

Basic geometry, time (clock) and uplift function

In [ ]:
xl = 10e3
yl = 10e3
nx = 101
ny = 101
tim = xr.DataArray(np.linspace(0,5e7,101), dims='tstep')
u = xr.DataArray(np.where(tim<2e7, 1e-3, 0), dims='tstep')


We create a simple xarray input dataset for the model

In [ ]:
# %create_setup spl_model -d
import xsimlab as xs

ds_in = xs.create_setup(
    model=spl_model,
    clocks={'tstep':tim,
           'time':[2e7,tim[-1]]},
    master_clock='tstep',
    input_vars={
        'grid__shape': [ny,nx],
        'grid__length': [yl,xl],
        'boundary__status': ['looped','looped','fixed_value','fixed_value'],
        'uplift__rate': u,
        'init_topography__seed': 123456789,
        'spl__k_coef': ('n', [1e-5,1e-7,1e-9,1e-11,1e-13]),
        'spl__area_exp': ('n', [0.45,0.9,1.35,1.8,2.25]),
        'spl__slope_exp': ('n', [1,2,3,4,5]),
        'spl__recession_exp': ('b', [1,2]),
        'spl__slope_exp_critical_stream_flow': ('n', [3/8,6/8,9/8,12/8,15/8]),#3/2,#1,
        'spl__daily_stream_power_exp': 0.75,
        'spl__threshold_erate': ('threshold', [1e-5,3e-5,1e-4,3e-4,1e-3,3e-3,1e-2]),
        'spl__PET': 5,
        'spl__soil_moisture_capacity': 1,
        'spl__storm_depth': 10,
        'spl__daily_rainfall_frequency': 0.6,
        'spl__storm_size': 1e12,
    },
    output_vars={'topography__elevation': 'tstep',
                'flow__receivers': 'time',
                'flow__nb_donors': 'time',
                'flow__donors': 'time',
                'drainage__area': 'time',
                'spl__streamflow_variability': 'time',
                'spl__response_time': 'time',
                'spl__streamflow_ratio': 'time'}
)


We run the batch of models and save the resuls in a zarr archive on disk

In [ ]:
import zarr
zgroup = zarr.group("Multiple_thresholds.zarr", overwrite=True)
with spl_model:
    ds_out = (ds_in.stack(batch=['threshold','n','b'])
              .xsimlab.run(store=zgroup, batch_dim='batch', parallel=True, scheduler='processes')
              .unstack('batch')
              .assign_coords({'threshold':ds_in.spl__threshold_erate/ds_in.uplift__rate.max()})
              .assign_coords({'n':ds_in.spl__slope_exp})
              .assign_coords({'b':ds_in.spl__recession_exp}))

We show the result of one model run as the maximum and final topography

In [ ]:
ds_out.topography__elevation.sel(tstep=(2e7,5e7)).sel(b=1,n=1).isel(threshold=-1).plot(col='tstep')

We reproduce one of the figures of the manuscript bu computing the ratio of mean topography at the end of the post-orogenic phase to the same thing at the end of the orogenic phase, for various values of the threshold (normalized by the uplift rate) and the parameters n and b

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
((ds_out.topography__elevation.isel(tstep=-1).mean(('x','y'))
  /ds_out.topography__elevation.sel(tstep=2e7).mean(('x','y')))
 .isel(b=0).plot.line(x='threshold',xscale='log',marker='o',ax=ax))
plt.gca().set_prop_cycle(None)
((ds_out.topography__elevation.isel(tstep=-1).mean(('x','y'))
  /ds_out.topography__elevation.sel(tstep=2e7).mean(('x','y')))
 .isel(b=1).plot.line(x='threshold',xscale='log',marker='+',ax=ax))
plt.gca().set_prop_cycle(None)
((ds_out.topography__elevation.isel(tstep=-1).mean(('x','y'))
  /ds_out.topography__elevation.sel(tstep=2e7).mean(('x','y')))
 .isel(b=1).plot.line(x='threshold',xscale='log',ax=ax))
ax.set_title('o b=1; + b=2')
ax.set_xlabel(r'$\epsilon_c/U$');
ax.set_ylabel(r'$R_\infty$');


We show the distribution of response time, tau, and srteam flow variability for all models

In [ ]:
fig,ax = plt.subplots(ncols=2, figsize=(12,4))
(ds_out.spl__response_time
             .isel(threshold=0)
             .isel(n=0)
             .isel(b=0)
             .isel(time=-1)
             .plot(ax=ax[0])
            )
(ds_out.spl__streamflow_variability
             .isel(threshold=0)
             .isel(n=0)
             .isel(b=0)
             .isel(time=-1)
             .plot(ax=ax[1])
            )
